### What is machine learning?
- Machine learning is a field of AI that gives computers the ability to learn from data,make decisions and predictions 

## Mini Project # 9.1- Handwritten Digit Recognition

### Data Prep, Training and Evaluation

In [ ]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
import time

In [ ]:
#Let us take a look at our digitals dataset
image=cv2.imread('images/digits.png')
gray=cv2.cvtColor(image,cv2.COLOR_BGR2GRAY)
small=cv2.pyrDown(image)
cv2.imshow('Digits Image',small)
cv2.waitKey(0)
cv2.destroyAllWindows()
 
#Split the image to 5000 cells,each 20X20 
#This gives us a 4-dim array: (50raws) x (100cols) x (20x20 each cell)
cells=[np.hsplit(row,100) for row in np.vsplit(gray,50)]
#Convert the List data type to Numpy Array of shape (50,100,20,20)
x=np.array(cells)
print('The shape of our cells array:'+ str(x.shape))

#Split the full data set into two segments
#One will be used for Training the model, the other as a test data set
train=x[:,:70].reshape(-1,400).astype(np.float32)# Size = (3500,400)
test=x[:,70:100].reshape(-1,400).astype(np.float32)# Size = (1500,400)

#Create labels for training and test data
k=[0,1,2,3,4,5,6,7,8,9]
train_labels=np.repeat(k,350)[:,np.newaxis]
test_labels=np.repeat(k,150)[:,np.newaxis]

#Initiate the kNN,train the data, then test it with test data for k=3
knn = cv2.ml.KNearest_create()
knn.train(train,cv2.ml.ROW_SAMPLE, train_labels)
ret, result, neighbors, distance = knn.findNearest(test, k=3)

#Now we check the accuracy of classification
#For that,compare the result with test_labels and check which are wrong
matches=result==test_labels
correct=np.count_nonzero(matches)
accuracy=correct*(100.0/result.size)
print('Accuracy is = ',accuracy,'%')





### Defining some functions we will use to prepare an input image

### Useful function:      cv2.copyMakeBorder(....)
- Get the input image, then compute h an w, if h>w, calculate the difference then do:
- 1. add the difference/2 on the left of w 
- 2. add the difference/2 on the right of w.
- OutputImage=cv2.copyMakeBorder(inputImage,top,bottom,left,right,borderType,const Scalar & value = Scalar())

- if w>h do it on both top and buttom 

In [ ]:
# Define our functions

def x_cord_contour(contours):
    # This function take a contour from findContours
    # it then outputs the x centroid coordinates
    if cv2.contourArea(contours) >= 10:
        M = cv2.moments(contours)
        return (int(M['m10']/M['m00']))
    else:
        return 0

def makeSquare(not_square):
    # This function takes an image and makes the dimenions square
    # It adds black pixels as the padding where needed
    
    BLACK = [0,0,0]
    img_dim = not_square.shape
    height = img_dim[0]
    width = img_dim[1]
    #print("Height = ", height, "Width = ", width)
    if (height == width):
        square = not_square
        return square
    else:
        doublesize = cv2.resize(not_square,(2*width, 2*height), interpolation = cv2.INTER_CUBIC)
        height = height * 2
        width = width * 2
        #print("New Height = ", height, "New Width = ", width)
        if (height > width):
            pad = int((height - width)/2)
            #print("Padding = ", pad)
            doublesize_square = cv2.copyMakeBorder(doublesize,0,0,pad,\
                                                   pad,cv2.BORDER_CONSTANT,value=BLACK)
        else:
            pad = int((width - height)/2)
            #print("Padding = ", pad)
            doublesize_square = cv2.copyMakeBorder(doublesize,pad,pad,0,0,\
                                                   cv2.BORDER_CONSTANT,value=BLACK)
    doublesize_square_dim = doublesize_square.shape
    #print("Sq Height = ", doublesize_square_dim[0], "Sq Width = ", doublesize_square_dim[1])
    return doublesize_square


def resize_to_pixel(dimensions, image):
    # This function then re-sizes an image to the specificied dimenions
    
    buffer_pix = 4
    dimensions  = dimensions - buffer_pix
    squared = image
    r = float(dimensions) / squared.shape[1]
    dim = (dimensions, int(squared.shape[0] * r))
    resized = cv2.resize(image, dim, interpolation = cv2.INTER_AREA)
    img_dim2 = resized.shape
    height_r = img_dim2[0]
    width_r = img_dim2[1]
    BLACK = [0,0,0]
    if (height_r > width_r):
        resized = cv2.copyMakeBorder(resized,0,0,0,1,cv2.BORDER_CONSTANT,value=BLACK)
    if (height_r < width_r):
        resized = cv2.copyMakeBorder(resized,1,0,0,0,cv2.BORDER_CONSTANT,value=BLACK)
    p = 2
    ReSizedImg = cv2.copyMakeBorder(resized,p,p,p,p,cv2.BORDER_CONSTANT,value=BLACK)
    img_dim = ReSizedImg.shape
    height = img_dim[0]
    width = img_dim[1]
    #print("Padded Height = ", height, "Width = ", width)
    return ReSizedImg

### Loading a new image, preprocessing it and classifying the digits


In [ ]:
image = cv2.imread('images/numbers.jpg')
gray = cv2.cvtColor(image,cv2.COLOR_BGR2GRAY)
cv2.imshow("image", image)
cv2.imshow("gray", gray)
cv2.waitKey(0)

# Blur image then find edges using Canny 
blurred = cv2.GaussianBlur(gray, (5, 5), 0)
cv2.imshow("blurred", blurred)
cv2.waitKey(0)

#Finding Canny edges
edged = cv2.Canny(blurred, 30, 150)
cv2.imshow("edged", edged)
cv2.waitKey(0)

# Fint Contours
_,contours, hierarchy = cv2.findContours(edged.copy(), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

#Sort out contours left to right by using their x cordinates
contours = sorted(contours,key = x_cord_contour, reverse = False)# x_cord_contour

# Create empty array to store entire number
full_number = []

# loop over the contours
for c in contours:
    # compute the bounding box for the rectangle
    (x, y, w, h) = cv2.boundingRect(c)    
    
    cv2.drawContours(image, contours, -1, (0,255,0), 3)
    cv2.imshow("Contours", image)

    if w >= 5 and h >= 25:
        roi = blurred[y:y + h, x:x + w]
        ret, roi = cv2.threshold(roi, 127, 255,cv2.THRESH_BINARY_INV)
        squared = makeSquare(roi)
        final = resize_to_pixel(20, squared)
        cv2.imshow("final", final)
        final_array = final.reshape((1,400))
        final_array = final_array.astype(np.float32)
        ret, result, neighbours, dist = knn.findNearest(final_array, k=1)
        number = str(int(float(result[0])))
        full_number.append(number)
        # draw a rectangle around the digit, the show what the
        # digit was classified as
        cv2.rectangle(image, (x, y), (x + w, y + h), (0, 0, 255), 2)
        cv2.putText(image, number, (x , y + 155),
            cv2.FONT_HERSHEY_COMPLEX, 2, (255, 0, 0), 2)
        cv2.imshow("image", image)
        cv2.waitKey(0) 
        
cv2.destroyAllWindows()
print ("The number is: " + ''.join(full_number))

## Mini Project # 9.2-Face Recognition – Unlock Your Computer With Your Face!

###  step1 - Create Training Data

In [1]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
import time

from os import listdir
from os.path import isfile, join

from PIL import Image
import glob

In [ ]:
#Load HAAR face classifier
face_classifier=cv2.CascadeClassifier('Haarcascades/haarcascade_frontalface_default.xml')

#Load function
def face_extractor(img):
    # Function detects faces and returns the cropped face
    # If no face detected, it returns the input image
    gray=cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
    faces=face_classifier.detectMultiScale(gray,1.3,5)
    
    if faces is ():
        return None
    
    #otherwise Crop all faces found
    for (x,y,w,h) in faces:
        cropped_face=img[y:y+h, x:x+w]
    return cropped_face

#Initialize Webcom
cap = cv2.VideoCapture(0)
count=0
#Collect 100 samples of your from webcam input
while True:
    ret,frame=cap.read()
    if face_extractor(frame) is not None:
        count+=1
        face=cv2.resize(face_extractor(frame),(200,200))
        face=cv2.cvtColor(face,cv2.COLOR_BGR2GRAY)
        
        #Save file in specified directory with unique name
        file_name_path='./faces/user/' + str(count) + '.jpg'
        cv2.imwrite(file_name_path, face)
        
        #Put count on images and display live count
        cv2.putText(face, str(count),(50,50),cv2.FONT_HERSHEY_COMPLEX,1,(0,255,0),2)
        cv2.imshow('Face Cropper',face)
       
    else:
        print('Face not found')
        pass
    
    if cv2.waitKey(1)==13 or count==100:#13 is the ENTER KEY
        break
        
cap.release()
cv2.destroyAllWindows()
print('Collecting Samples Complete')


### Step 2 - Train Model

In [ ]:
#Creating arrays for training data and labels
Training_Data, Labels=[],[]

#Loading training images with pillow library
from PIL import Image
import glob

image_list = []
for filename in glob.glob('./faces/user/*.jpg'): #assuming gif
    im=Image.open(filename)
    #im.show()
    image_list.append(np.asarray(im,dtype=np.int))
    Labels.append(1)
#Labels = np.asarray(Labels, dtype=np.int)    

  
for filename in glob.glob('./faces1/user/*.jpg'): #assuming gif
    im=Image.open(filename)
    #im.show()
    image_list.append(np.asarray(im,dtype=np.int))
    Labels.append(2)  
    
Labels = np.asarray(Labels, dtype=np.int)


#Initialize facial recognizer
model=cv2.face.LBPHFaceRecognizer_create()



#Let's train our model
model.train((image_list),(Labels))
model.save("face-trainner.yml")
print('model trained sucessfully')


### Step 3 - Run Our Facial Recognition

In [ ]:
face_classifier=cv2.CascadeClassifier('Haarcascades/haarcascade_frontalface_default.xml')
model=cv2.face.LBPHFaceRecognizer_create()
model.read("face-trainner.yml")


def face_detector(img,size=0.5):
    #Convert image to grayscale
    gray=cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
    faces=face_classifier.detectMultiScale(gray,1.3,5)
    if faces is():
        return img, []
    for (x,y,w,h) in faces:
        cv2.rectangle(img,(x,y),(x+w, y+h),(0,255,255),2)
        roi=img[y:y+h, x:x+w]
        roi=cv2.resize(roi,(200,200))
    return img, roi

#Open Webcam
cap = cv2.VideoCapture(0)
while True:
    ret,frame=cap.read()
    image,face=face_detector(frame)
    try:
        face=cv2.cvtColor(face,cv2.COLOR_BGR2GRAY)
        # Pass face to prediction model
        # "results" comprises of a tuple containing the label and the confidence value
        results=model.predict(face)
        id_, conf = model.predict(face)
        print(id_)
        print(conf)
    
        if results[1]<500:
            confidence = int(100*(1-(results[1])/400)) 
            display_string=str(confidence)+'% Confident is is User'
        cv2.putText(image, display_string, (100, 120), cv2.FONT_HERSHEY_COMPLEX, 1, (255,120,150), 2)    

        if confidence > 75 and id_==1:
             cv2.putText(image, "Cesar-Unlocked", (250, 450), cv2.FONT_HERSHEY_COMPLEX, 1, (0,255,0), 2)
             cv2.imshow('Face Recognition', image )
            
        elif confidence > 75 and id_==2:
             cv2.putText(image, "xxxx-Unlocked", (250, 450), cv2.FONT_HERSHEY_COMPLEX, 1, (0,255,0), 2)
             cv2.imshow('Face Recognition', image )           
            
        else:
             cv2.putText(image, "Locked", (250, 450), cv2.FONT_HERSHEY_COMPLEX, 1, (0,0,255), 2)
             cv2.imshow('Face Recognition', image )

    except:
        cv2.putText(image, "No Face Found", (220, 120) , cv2.FONT_HERSHEY_COMPLEX, 1, (0,0,255), 2)
        cv2.putText(image, "Locked", (250, 450), cv2.FONT_HERSHEY_COMPLEX, 1, (0,0,255), 2)
        cv2.imshow('Face Recognition', image )
        pass
        
    if cv2.waitKey(1) == 13: #13 is the Enter Key
        break
        
cap.release()
cv2.destroyAllWindows()  


In [ ]:
'''''
import cv2
import os
import numpy as np
from PIL import Image
import pickle

BASE_DIR = os.path.dirname(os.path.abspath(__file__))#__file__
image_dir = os.path.join(BASE_DIR, "images1")

face_cascade = cv2.CascadeClassifier('cascades/data/haarcascade_frontalface_alt2.xml')
recognizer = cv2.face.LBPHFaceRecognizer_create()

current_id = 0
label_ids = {}
y_labels = []
x_train = []

for root, dirs, files in os.walk(image_dir):
    for file in files:
        if file.endswith("png") or file.endswith("jpg"):
            path = os.path.join(root, file)
            label = os.path.basename(root).replace(" ", "-").lower()
            #print(label, path)
            if not label in label_ids:
                label_ids[label] = current_id
                current_id += 1
            id_ = label_ids[label]
            #print(label_ids)
            #y_labels.append(label) # some number
            #x_train.append(path) # verify this image, turn into a NUMPY arrray, GRAY
            pil_image = Image.open(path).convert("L") # grayscale
            size = (550, 550)
            final_image = pil_image.resize(size, Image.ANTIALIAS)
            image_array = np.array(final_image, "uint8")
            #print(image_array)
            faces = face_cascade.detectMultiScale(image_array, scaleFactor=1.5, minNeighbors=5)

            for (x,y,w,h) in faces:
                roi = image_array[y:y+h, x:x+w]
                x_train.append(roi)
                y_labels.append(id_)


#print(y_labels)
#print(x_train)

with open("pickles/face-labels.pickle", 'wb') as f:
    pickle.dump(label_ids, f)

recognizer.train(x_train, np.array(y_labels))
recognizer.save("recognizers/face-trainner.yml")


#--------------------------------------------------------------------------
import numpy as np
import cv2
import pickle

face_cascade = cv2.CascadeClassifier('cascades/data/haarcascade_frontalface_alt2.xml')
eye_cascade = cv2.CascadeClassifier('cascades/data/haarcascade_eye.xml')
smile_cascade = cv2.CascadeClassifier('cascades/data/haarcascade_smile.xml')


recognizer = cv2.face.LBPHFaceRecognizer_create()
recognizer.read("./recognizors/face-trainner.yml")

labels = {"person_name": 1}
with open("pickles/face-labels.pickle", 'rb') as f:
	og_labels = pickle.load(f)
	labels = {v:k for k,v in og_labels.items()}

cap = cv2.VideoCapture(0)

while(True):
    # Capture frame-by-frame
    ret, frame = cap.read()
    gray  = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    faces = face_cascade.detectMultiScale(gray, scaleFactor=1.5, minNeighbors=5)
    for (x, y, w, h) in faces:
    	#print(x,y,w,h)
    	roi_gray = gray[y:y+h, x:x+w] #(ycord_start, ycord_end)
    	roi_color = frame[y:y+h, x:x+w]

    	# recognize? deep learned model predict keras tensorflow pytorch scikit learn
    	id_, conf = recognizer.predict(roi_gray)
    	if conf>=4 and conf <= 85:
    		#print(5: #id_)
    		#print(labels[id_])
    		font = cv2.FONT_HERSHEY_SIMPLEX
    		name = labels[id_]
    		color = (255, 255, 255)
    		stroke = 2
    		cv2.putText(frame, name, (x,y), font, 1, color, stroke, cv2.LINE_AA)

    	img_item = "7.png"
    	cv2.imwrite(img_item, roi_color)

    	color = (255, 0, 0) #BGR 0-255 
    	stroke = 2
    	end_cord_x = x + w
    	end_cord_y = y + h
    	cv2.rectangle(frame, (x, y), (end_cord_x, end_cord_y), color, stroke)
    	#subitems = smile_cascade.detectMultiScale(roi_gray)
    	#for (ex,ey,ew,eh) in subitems:
    	#	cv2.rectangle(roi_color,(ex,ey),(ex+ew,ey+eh),(0,255,0),2)
    # Display the resulting frame
    cv2.imshow('frame',frame)
    if cv2.waitKey(20) & 0xFF == ord('q'):
        break

# When everything done, release the capture
cap.release()
cv2.destroyAllWindows()

'''''